In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import logging

from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
logging.getLogger('prophet').setLevel(logging.WARNING)
logging.getLogger('cmdstanpy').setLevel(logging.DEBUG)
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)

## Preenchimento de dados faltantes para Data

In [ ]:
df_agregado_mensal = pd.read_csv('/content/drive/MyDrive/data/Agregado/Dados_Agregados_Mensal.csv')

In [ ]:
df_agregado_mensal.head(3)

,Instituição,Cod.IBGE,UF,População,Mês,Conta,Valor,Ano
0,Prefeitura Municipal de Estância - SE,2802106,SE,68405,1,IPTU,4208.41,2016
1,Prefeitura Municipal de Estância - SE,2802106,SE,68405,1,ISS,652846.87,2016
2,Prefeitura Municipal de Estância - SE,2802106,SE,68405,1,ITBI,62185.36,2016


In [ ]:
df_agregado_mensal['Data'] = df_agregado_mensal['Mês'].astype(str).str.zfill(2) + '/' + df_agregado_mensal['Ano'].astype(str)
df_agregado_mensal['Data'] = pd.to_datetime(df_agregado_mensal['Data'], format='%m/%Y')

In [ ]:
df_agregado_mensal.head(3)

,Instituição,Cod.IBGE,UF,População,Mês,Conta,Valor,Ano,Data
0,Prefeitura Municipal de Estância - SE,2802106,SE,68405,1,IPTU,4208.41,2016,2016-01-01
1,Prefeitura Municipal de Estância - SE,2802106,SE,68405,1,ISS,652846.87,2016,2016-01-01
2,Prefeitura Municipal de Estância - SE,2802106,SE,68405,1,ITBI,62185.36,2016,2016-01-01


In [ ]:
date_range = pd.date_range(start='2016-01-01', end='2023-12-01', freq='MS')
novas_linhas = []

for municipio in df_agregado_mensal['Instituição'].unique():
  for conta in df_agregado_mensal['Conta'].unique():
    df_slice = df_agregado_mensal.loc[(df_agregado_mensal['Instituição'] == municipio) & (df_agregado_mensal['Conta'] == conta)]
    datas_faltando = date_range.difference(df_slice['Data'])

    for data in datas_faltando:
      novas_linhas.append({
          'Instituição': municipio,
          'Conta': conta,
          'Data': data
      })

In [ ]:
df_novas_linhas = pd.DataFrame(novas_linhas) #Linhas Faltantes
df_novas_linhas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Instituição  391 non-null    object        
 1   Conta        391 non-null    object        
 2   Data         391 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 9.3+ KB


In [ ]:
df_preenchido = pd.concat([df_agregado_mensal, df_novas_linhas], ignore_index=True)
df_preenchido = df_preenchido.sort_values(by=['Instituição', 'Conta', 'Data']).reset_index(drop=True)

In [ ]:
df_preenchido.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5760 entries, 0 to 5759
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Instituição  5760 non-null   object        
 1   Cod.IBGE     5369 non-null   float64       
 2   UF           5369 non-null   object        
 3   População    5369 non-null   float64       
 4   Mês          5369 non-null   float64       
 5   Conta        5760 non-null   object        
 6   Valor        5369 non-null   float64       
 7   Ano          5369 non-null   float64       
 8   Data         5760 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(5), object(3)
memory usage: 405.1+ KB


In [ ]:
# Preencher Cod.IBGE e UF baseado na Instituição
for instituicao in df_preenchido['Instituição'].unique():
    # Extrair Cod.IBGE e UF da primeira ocorrência para cada instituição
    cod_ibge = df_preenchido.loc[df_preenchido['Instituição'] == instituicao, 'Cod.IBGE'].dropna().values[0]
    uf = instituicao.split('-')[-1].strip()

    # Preencher Cod.IBGE e UF onde estiver NaN
    df_preenchido.loc[(df_preenchido['Instituição'] == instituicao) & (df_preenchido['Cod.IBGE'].isna()), 'Cod.IBGE'] = cod_ibge
    df_preenchido.loc[(df_preenchido['Instituição'] == instituicao) & (df_preenchido['UF'].isna()), 'UF'] = uf

# Preencher Mês e Ano a partir da coluna Data
df_preenchido['Mês'] = df_preenchido['Data'].dt.month
df_preenchido['Ano'] = df_preenchido['Data'].dt.year

# Preencher os NaN de População e Valor com 0
df_preenchido['População'] = df_preenchido['População'].fillna(0)
df_preenchido['Valor'] = df_preenchido['Valor'].fillna(0)

In [ ]:
df_preenchido.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5760 entries, 0 to 5759
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Instituição  5760 non-null   object        
 1   Cod.IBGE     5760 non-null   float64       
 2   UF           5760 non-null   object        
 3   População    5760 non-null   float64       
 4   Mês          5760 non-null   int32         
 5   Conta        5760 non-null   object        
 6   Valor        5760 non-null   float64       
 7   Ano          5760 non-null   int32         
 8   Data         5760 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int32(2), object(3)
memory usage: 360.1+ KB


## Preenchimento de dados faltantes para População

In [ ]:
df_agrupado_pop = df_preenchido.groupby(['Instituição', 'Ano'])['População'].max().reset_index()

In [ ]:
df_agrupado_pop.replace(0, np.nan, inplace=True)

In [ ]:
df_agrupado_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Instituição  160 non-null    object 
 1   Ano          160 non-null    int32  
 2   População    155 non-null    float64
dtypes: float64(1), int32(1), object(1)
memory usage: 3.2+ KB


In [ ]:
# Segunda Alternativa de Preenchimento
from sklearn.linear_model import LinearRegression

for municipio in df_agrupado_pop['Instituição'].unique():
  df_municipio = df_agrupado_pop[df_agrupado_pop['Instituição'] == municipio]

  df_not_null = df_municipio.dropna(subset=['População'])
  df_missing = df_municipio[df_municipio['População'].isna()]

  if len(df_missing) != 0:
    X_train = df_not_null[['Ano']]  # Feature
    y_train = df_not_null['População']

    model = LinearRegression()
    model.fit(X_train, y_train)

    df_agrupado_pop.loc[(df_agrupado_pop['População'].isna()) & (df_agrupado_pop['Instituição'] == municipio), 'População'] = model.predict(df_missing[['Ano']]).astype(int)

In [ ]:
for municipio in df_preenchido['Instituição'].unique():
  for ano in df_preenchido['Ano'].unique():
    df_preenchido.loc[(df_preenchido['População'] == 0) & (df_preenchido['Instituição'] == municipio) & (df_preenchido['Ano'] == ano), 'População'] = df_agrupado_pop.loc[(df_agrupado_pop['Instituição'] == municipio) & (df_agrupado_pop['Ano'] == ano), 'População'].values[0]

In [ ]:
df_agrupado_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Instituição  160 non-null    object 
 1   Ano          160 non-null    int32  
 2   População    160 non-null    float64
dtypes: float64(1), int32(1), object(1)
memory usage: 3.2+ KB


In [ ]:
df_preenchido['Instituição'] = df_preenchido['Instituição'].str.replace('Prefeitura Municipal de ', '', regex=False)
df_preenchido['Instituição'] = df_preenchido['Instituição'].str.replace(r' - [A-Z]{2}', '', regex=True)

In [ ]:
df_preenchido.head()

,Instituição,Cod.IBGE,UF,População,Mês,Conta,Valor,Ano,Data
0,Armação dos Búzios,3300233.0,RJ,31067.0,1,IPTU,2658874.7,2016,2016-01-01
1,Armação dos Búzios,3300233.0,RJ,31067.0,2,IPTU,6301062.8,2016,2016-02-01
2,Armação dos Búzios,3300233.0,RJ,31067.0,3,IPTU,1357259.6,2016,2016-03-01
3,Armação dos Búzios,3300233.0,RJ,31067.0,4,IPTU,1171113.9,2016,2016-04-01
4,Armação dos Búzios,3300233.0,RJ,31067.0,5,IPTU,429706.8,2016,2016-05-01


## Preenchimento de dados faltantes para os Impostos

In [ ]:
for municipio in df_preenchido['Instituição'].unique():
  for imposto in df_preenchido['Conta'].unique():
    df_filtered = df_preenchido[(df_preenchido['Instituição'] == municipio) & (df_preenchido['Conta'] == imposto)][['Data', 'Valor']]
    df_filtered.reset_index(drop=True, inplace=True)
    df_filtered.replace(0, np.nan, inplace=True)
    df_filtered.rename(columns={'Data':'ds', 'Valor':'y'}, inplace=True)

    if(df_filtered['y'].isna().any()):
      model = Prophet()
      model.fit(df_filtered)

      future = model.make_future_dataframe(periods=0, freq='MS')
      forecast = model.predict(future)

      forecast['yhat'] = forecast.apply(lambda row: row['yhat'] if row['yhat'] > 0 else row['yhat_upper'], axis=1)
      forecast = forecast[['ds', 'yhat']]

      indices_nan = df_filtered[df_filtered['y'].isna()].index
      df_filtered.loc[indices_nan, 'y'] = forecast.loc[indices_nan, 'yhat'].values

      df_preenchido.loc[(df_preenchido['Instituição'] == municipio) & (df_preenchido['Conta'] == imposto), 'Valor'] = np.round(df_filtered['y'].values)

In [ ]:
df_preenchido.to_csv('/content/drive/MyDrive/data/Preenchido/Dados_Preenchidos.csv', index=False)